In [ ]:
import pygame
import sys
import random
import math
import numpy as np

# Inicialización de Pygame
pygame.init()

# Dimensiones de la ventana
WIDTH, HEIGHT = 800, 600
WINDOW = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Sistema Multiagente de Gestión de Residuos Urbanos")

# Colores
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREEN = (0, 200, 0)
BLUE = (0, 100, 255)
RED = (200, 0, 0)
YELLOW = (255, 255, 0)
BROWN = (139, 69, 19)
ORANGE = (255, 165, 0)
GRAY = (128, 128, 128)

# FPS
FPS = 60
clock = pygame.time.Clock()

# Definición del Entorno Urbano
class CityEnvironment:
    def __init__(self, num_points):
        self.num_points = num_points
        self.waste_points = self.generate_waste_points(num_points)
        self.centers = self.generate_centers()
        self.grid_size = 20  # Tamaño de la cuadrícula para el algoritmo A*
        self.central_station = CentralStation()

    def generate_waste_points(self, num_points):
        waste_points = []
        for _ in range(num_points):
            point = WastePoint(
                x=random.randint(50, WIDTH - 50),
                y=random.randint(50, HEIGHT - 50),
                waste_type=random.choice(['orgánico', 'inorgánico', 'otro']),
                weight=random.randint(1, 5)  # Peso entre 1 y 5 unidades
            )
            waste_points.append(point)
        return waste_points

    def generate_centers(self):
        centers = [
            TreatmentCenter(x=100, y=100, waste_type='orgánico'),
            TreatmentCenter(x=400, y=100, waste_type='inorgánico'),
            TreatmentCenter(x=700, y=100, waste_type='otro')
        ]
        return centers

    def draw(self, window):
        for point in self.waste_points:
            point.draw(window)
        for center in self.centers:
            center.draw(window)
        self.central_station.draw(window)

# Puntos de Residuos
class WastePoint:
    def __init__(self, x, y, waste_type, weight):
        self.x = x
        self.y = y
        self.waste_type = waste_type
        self.weight = weight
        self.collected = False
        self.reserved = False

    def draw(self, window):
        if not self.collected:
            if self.reserved:
                color = GRAY  # Residuos reservados
            else:
                if self.waste_type == 'orgánico':
                    color = GREEN
                elif self.waste_type == 'inorgánico':
                    color = BLUE
                else:
                    color = YELLOW
            pygame.draw.circle(window, color, (self.x, self.y), 5)

# Centros de Tratamiento
class TreatmentCenter:
    def __init__(self, x, y, waste_type):
        self.x = x
        self.y = y
        self.waste_type = waste_type
        self.capacity = 100  # Capacidad máxima

    def draw(self, window):
        # Dibujar una casa sencilla
        pygame.draw.rect(window, RED, (self.x - 10, self.y, 20, 20))  # Cuerpo de la casa
        pygame.draw.polygon(window, BROWN, [(self.x - 15, self.y), (self.x + 15, self.y), (self.x, self.y - 20)])  # Techo
        # Dibujar el texto del tipo de residuo
        font = pygame.font.SysFont('Arial', 14)
        text_surface = font.render(self.waste_type.capitalize(), True, BLACK)
        window.blit(text_surface, (self.x - 20, self.y - 30))

# Estación Central
class CentralStation:
    def __init__(self):
        self.transport_agent_state = 'waiting'  # 'waiting', 'moving_to_center', 'returning'
        self.transport_agent_position = (WIDTH // 2, HEIGHT // 2)  # Posición inicial del camión

    def update_state(self, state):
        self.transport_agent_state = state

    def get_state(self):
        return self.transport_agent_state

    def draw(self, window):
        pass  # No es necesario dibujar la estación central

# Clase base para agentes
class Agent:
    def __init__(self, x, y, environment, name):
        self.x = x
        self.y = y
        self.environment = environment
        self.path = []
        self.speed = 2
        self.name = name

    def move(self):
        if self.path:
            next_pos = self.path[0]
            dx = next_pos[0] - self.x
            dy = next_pos[1] - self.y
            dist = math.hypot(dx, dy)
            if dist < self.speed:
                self.x, self.y = next_pos
                self.path.pop(0)
            else:
                self.x += dx / dist * self.speed
                self.y += dy / dist * self.speed

    def draw(self, window):
        pass

# Agente de Recolección
class CollectionAgent(Agent):
    def __init__(self, x, y, environment, name):
        super().__init__(x, y, environment, name)
        self.state = 'idle'  # 'idle', 'moving_to_waste', 'collecting', 'moving_to_truck', 'delivering', 'waiting_for_truck'
        self.target = None
        self.collected_waste = []

    def perceive(self):
        # Percibe los puntos de residuos no recolectados
        self.visible_waste = [point for point in self.environment.waste_points if not point.collected]
        # Percibe el estado del agente de transporte
        self.transport_state = self.environment.central_station.get_state()
        # Percibe la posición del camión
        self.transport_position = self.environment.central_station.transport_agent_position

    def decide(self):
        if self.state == 'idle' and self.visible_waste:
            # Selecciona el residuo más cercano no reservado
            unreserved_waste = [p for p in self.visible_waste if not p.reserved]
            if unreserved_waste:
                self.target = min(unreserved_waste, key=lambda p: self.distance((self.x, self.y), (p.x, p.y)))
                self.target.reserved = True  # Reserva el punto de residuo
                # Calcula la ruta utilizando A*
                self.path = self.a_star_search((self.x, self.y), (self.target.x, self.target.y))
                self.state = 'moving_to_waste'

        elif self.state == 'moving_to_waste' and not self.path:
            self.state = 'collecting'

        elif self.state == 'collecting':
            self.target.collected = True
            self.target.reserved = False  # Libera la reserva
            self.collected_waste.append(self.target)
            print(f"{self.name} recogió un residuo de tipo {self.target.waste_type}.")  # Depuración
            self.target = None
            # Después de recolectar, ir al camión si está disponible
            if self.transport_state == 'waiting':
                truck_position = self.environment.central_station.transport_agent_position
                self.path = self.a_star_search((self.x, self.y), truck_position)
                self.state = 'moving_to_truck'
            else:
                self.state = 'waiting_for_truck'

        elif self.state == 'waiting_for_truck':
            # Espera hasta que el camión esté disponible
            if self.transport_state == 'waiting':
                truck_position = self.environment.central_station.transport_agent_position
                self.path = self.a_star_search((self.x, self.y), truck_position)
                self.state = 'moving_to_truck'

        elif self.state == 'moving_to_truck' and not self.path:
            
            # Solo entrega si el camión puede recibir residuos
            if transport_agent.current_load < transport_agent.capacity:
                self.state = 'delivering'
            else:
                self.state = 'waiting_for_truck'

        elif self.state == 'delivering':
            # Entrega los residuos al camión
            transport_agent.receive_waste(self.collected_waste)
            print(f"{self.name} entregó {len(self.collected_waste)} residuos al camión.")
            self.collected_waste = []
            self.state = 'idle'

    def act(self):
        if self.state in ['moving_to_waste', 'moving_to_truck']:
            self.move()

    def draw(self, window):
        # Dibujar una figura humana sencilla
        pygame.draw.circle(window, BLACK, (int(self.x), int(self.y - 10)), 5)  # Cabeza
        pygame.draw.line(window, BLACK, (int(self.x), int(self.y - 5)), (int(self.x), int(self.y + 10)), 2)  # Cuerpo
        pygame.draw.line(window, BLACK, (int(self.x), int(self.y + 10)), (int(self.x - 5), int(self.y + 20)), 2)  # Pierna izquierda
        pygame.draw.line(window, BLACK, (int(self.x), int(self.y + 10)), (int(self.x + 5), int(self.y + 20)), 2)  # Pierna derecha
        pygame.draw.line(window, BLACK, (int(self.x), int(self.y)), (int(self.x - 5), int(self.y + 5)), 2)  # Brazo izquierdo
        pygame.draw.line(window, BLACK, (int(self.x), int(self.y)), (int(self.x + 5), int(self.y + 5)), 2)  # Brazo derecho

    def distance(self, pos1, pos2):
        return math.hypot(pos1[0]-pos2[0], pos1[1]-pos2[1])

    def a_star_search(self, start, goal):
        # Implementación simplificada del algoritmo A*
        grid_size = self.environment.grid_size
        start_grid = (int(start[0] // grid_size), int(start[1] // grid_size))
        goal_grid = (int(goal[0] // grid_size), int(goal[1] // grid_size))

        frontier = []
        frontier.append(start_grid)
        came_from = {}
        cost_so_far = {}
        came_from[start_grid] = None
        cost_so_far[start_grid] = 0

        while frontier:
            current = frontier.pop(0)

            if current == goal_grid:
                break

            neighbors = self.get_neighbors(current)
            for next in neighbors:
                new_cost = cost_so_far[current] + self.distance_grid(current, next)
                if next not in cost_so_far or new_cost < cost_so_far[next]:
                    cost_so_far[next] = new_cost
                    priority = new_cost + self.heuristic(goal_grid, next)
                    frontier.append(next)
                    came_from[next] = current

        # Reconstruir el camino
        current = goal_grid
        path = []
        while current != start_grid:
            x = current[0] * grid_size + grid_size // 2
            y = current[1] * grid_size + grid_size // 2
            path.insert(0, (x, y))
            current = came_from[current]
            if current is None:
                break
        return path

    def get_neighbors(self, grid_pos):
        neighbors = []
        dirs = [(-1,0),(1,0),(0,-1),(0,1)]
        for dir in dirs:
            nx = grid_pos[0] + dir[0]
            ny = grid_pos[1] + dir[1]
            if 0 <= nx * self.environment.grid_size < WIDTH and 0 <= ny * self.environment.grid_size < HEIGHT:
                neighbors.append((nx, ny))
        return neighbors

    def heuristic(self, a, b):
        return abs(a[0]-b[0]) + abs(a[1]-b[1])

    def distance_grid(self, a, b):
        return 1  # En cuadrícula, cada movimiento tiene un coste uniforme

# Agente de Transporte
class TransportAgent(Agent):
    def __init__(self, x, y, environment, capacity):
        super().__init__(x, y, environment, "TransportAgent")
        self.state = 'waiting'  # 'waiting', 'moving_to_center', 'returning'
        self.capacity = capacity
        self.current_load = 0  # Número de residuos recogidos
        self.collected_waste = []
        self.environment.central_station.transport_agent_position = (self.x, self.y)  # Actualizar posición en la estación central
        self.target_centers = []
        self.current_center_index = 0

    def perceive(self):
        # Actualiza el estado en la estación central
        self.environment.central_station.update_state(self.state)
        self.environment.central_station.transport_agent_position = (self.x, self.y)
        
        if self.state == 'waiting' and self.current_load == self.capacity:
            # Inicializa la secuencia de visitas a los centros de tratamiento
            self.target_centers = self.environment.centers
            self.current_center_index = 0
            self.visit_next_center()
            
        elif self.state == 'moving_to_center' and not self.path:
            self.state = 'delivering'

        elif self.state == 'delivering':
            # Entrega los residuos al agente de clasificación en el centro correspondiente
            current_center = self.target_centers[self.current_center_index]
            classification_agents[self.current_center_index].receive_waste([waste for waste in self.collected_waste if waste.waste_type == current_center.waste_type])
            self.collected_waste = [waste for waste in self.collected_waste if waste.waste_type != current_center.waste_type]
            print(f"Camión entregó residuos al centro de tratamiento {current_center.waste_type}.")
            self.current_load = len(self.collected_waste)

            # Si quedan residuos, visitar el siguiente centro, de lo contrario regresar al centro
            if self.current_load > 0 and self.current_center_index < len(self.target_centers) - 1:
                self.current_center_index += 1
                self.visit_next_center()
            else:
                self.path = self.a_star_search((self.x, self.y), (WIDTH // 2, HEIGHT // 2))
                self.state = 'returning'

        elif self.state == 'returning' and not self.path:
            self.state = 'waiting'

    def receive_waste(self, waste):
        # Agrega los residuos a su carga si no excede la capacidad
        total_items = self.current_load + len(waste)
        if total_items <= self.capacity:
            self.collected_waste.extend(waste)
            self.current_load = total_items
            for w in waste:
                print(f"Debug: Camión recibió residuo de tipo {w.waste_type}.")  # Depuración
            print(f"Camión recibió {len(waste)} residuos. Carga actual: {self.current_load}/{self.capacity}")
            # Cambia el estado a "moverse al centro" si alcanza la capacidad
            if self.current_load == self.capacity:
                self.state = 'moving_to_center'
                self.visit_next_center()
        else:
            print("El camión ha alcanzado su capacidad máxima y no puede cargar más residuos.")

    def act(self):
        if self.state in ['moving_to_center', 'returning']:
            self.move()

    def visit_next_center(self):
        # Moverse al siguiente centro de tratamiento
        next_center = self.target_centers[self.current_center_index]
        self.path = self.a_star_search((self.x, self.y), (next_center.x, next_center.y))
        self.state = 'moving_to_center'

    def draw(self, window):
        # Dibujar un camión sencillo
        pygame.draw.rect(window, ORANGE, (int(self.x) - 10, int(self.y) - 5, 20, 10))  # Cuerpo del camión
        pygame.draw.rect(window, GRAY, (int(self.x) - 15, int(self.y) - 5, 5, 10))  # Cabina
        pygame.draw.circle(window, BLACK, (int(self.x) - 5, int(self.y) + 5), 3)  # Rueda trasera
        pygame.draw.circle(window, BLACK, (int(self.x) + 5, int(self.y) + 5), 3)  # Rueda delantera

    def distance(self, pos1, pos2):
        return math.hypot(pos1[0]-pos2[0], pos1[1]-pos2[1])

    def a_star_search(self, start, goal):
        # Implementación simplificada del algoritmo A*
        grid_size = self.environment.grid_size
        start_grid = (int(start[0] // grid_size), int(start[1] // grid_size))
        goal_grid = (int(goal[0] // grid_size), int(goal[1] // grid_size))

        frontier = []
        frontier.append(start_grid)
        came_from = {}
        cost_so_far = {}
        came_from[start_grid] = None
        cost_so_far[start_grid] = 0

        while frontier:
            current = frontier.pop(0)

            if current == goal_grid:
                break

            neighbors = self.get_neighbors(current)
            for next in neighbors:
                new_cost = cost_so_far[current] + self.distance_grid(current, next)
                if next not in cost_so_far or new_cost < cost_so_far[next]:
                    cost_so_far[next] = new_cost
                    priority = new_cost + self.heuristic(goal_grid, next)
                    frontier.append(next)
                    came_from[next] = current

        # Reconstruir el camino
        current = goal_grid
        path = []
        while current != start_grid:
            x = current[0] * grid_size + grid_size // 2
            y = current[1] * grid_size + grid_size // 2
            path.insert(0, (x, y))
            current = came_from[current]
            if current is None:
                break
        return path

    def get_neighbors(self, grid_pos):
        neighbors = []
        dirs = [(-1,0),(1,0),(0,-1),(0,1)]
        for dir in dirs:
            nx = grid_pos[0] + dir[0]
            ny = grid_pos[1] + dir[1]
            if 0 <= nx * self.environment.grid_size < WIDTH and 0 <= ny * self.environment.grid_size < HEIGHT:
                neighbors.append((nx, ny))
        return neighbors

    def heuristic(self, a, b):
        return abs(a[0]-b[0]) + abs(a[1]-b[1])

    def distance_grid(self, a, b):
        return 1

# Agentes de Clasificación
class ClassificationAgent:
    def __init__(self, name):
        self.received_waste = []
        self.name = name

    def receive_waste(self, waste_list):
        self.received_waste.extend(waste_list)

    def classify_waste(self):
        if self.received_waste:
            for waste in self.received_waste:
                print(f"{self.name} clasificando residuo {waste.waste_type} de peso {waste.weight} en posición ({waste.x}, {waste.y})")
            self.received_waste = []

    def draw(self, window):
        pass

# Inicialización del entorno y agentes
environment = CityEnvironment(num_points=20)
collection_agents = []
for i in range(3):  # Tres agentes de recolección
    agent = CollectionAgent(
        x=random.randint(50, WIDTH - 50),
        y=random.randint(50, HEIGHT - 50),
        environment=environment,
        name=f"CollectionAgent_{i + 1}"
    )
    collection_agents.append(agent)

# Agente de Transporte ubicado en el centro del mapa
transport_agent = TransportAgent(
    x=WIDTH // 2,
    y=HEIGHT // 2,
    environment=environment,
    capacity=5  # Capacidad máxima de residuos
)

classification_agents = [
    ClassificationAgent(name="ClassificationAgent_Orgánico"),
    ClassificationAgent(name="ClassificationAgent_Inorgánico"),
    ClassificationAgent(name="ClassificationAgent_Otro")
]

# Bucle principal de la simulación
def main():
    run = True
    while run:
        clock.tick(FPS)
        WINDOW.fill(WHITE)
        environment.draw(WINDOW)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False

        # Actualización de agentes de recolección
        for agent in collection_agents:
            agent.perceive()
            agent.decide()
            agent.act()
            agent.draw(WINDOW)

        # Actualización del agente de transporte
        transport_agent.perceive()
        transport_agent.act()
        transport_agent.draw(WINDOW)

        # Agentes de clasificación procesan los residuos
        for classification_agent in classification_agents:
            classification_agent.classify_waste()

        pygame.display.update()

    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()


pygame 2.6.1 (SDL 2.28.4, Python 3.11.10)
Hello from the pygame community. https://www.pygame.org/contribute.html
CollectionAgent_1 recogió un residuo de tipo otro.
CollectionAgent_2 recogió un residuo de tipo otro.
CollectionAgent_3 recogió un residuo de tipo otro.
Debug: Camión recibió residuo de tipo otro.
Camión recibió 1 residuos. Carga actual: 1/5
CollectionAgent_2 entregó 1 residuos al camión.
Debug: Camión recibió residuo de tipo otro.
Camión recibió 1 residuos. Carga actual: 2/5
CollectionAgent_1 entregó 1 residuos al camión.
CollectionAgent_2 recogió un residuo de tipo orgánico.
CollectionAgent_1 recogió un residuo de tipo otro.
Debug: Camión recibió residuo de tipo otro.
Camión recibió 1 residuos. Carga actual: 3/5
CollectionAgent_3 entregó 1 residuos al camión.
Debug: Camión recibió residuo de tipo orgánico.
Camión recibió 1 residuos. Carga actual: 4/5
CollectionAgent_2 entregó 1 residuos al camión.
Debug: Camión recibió residuo de tipo otro.
Camión recibió 1 residuos. Carg

IndexError: list index out of range

: 